In [ ]:
import numpy as np

def calculate_pmi(word, class_label, dataset):
    word_count = sum(1 for doc in dataset if word in doc)
    class_count = sum(1 for doc in dataset if doc[1] == class_label)
    word_class_count = sum(1 for doc in dataset if word in doc and doc[1] == class_label)
    total_docs = len(dataset)

    p_word = word_count / total_docs
    p_class = class_count / total_docs
    p_word_class = word_class_count / total_docs
    print(p_word, p_class, p_word_class)

    if p_word == 0 or p_class == 0 or p_word_class == 0:
        return 0

    pmi = np.log2(p_word_class / (p_word * p_class))

    return pmi

# Example dataset (list of tuples where each tuple contains the text and class label)
dataset = [
    ("I loved the movie!", "positive"),
    ("The plot was confusing.", "negative"),
    ("The acting was great.", "positive"),
    ("It was a boring film.", "negative"),
    ("The movie was amazing!", "positive")
]

# Specify the words and classes you want to analyze
words = ["loved", "acting"]
classes = ["positive", "negative"]

# Calculate PMI for each word-class combination
for word in words:
    for class_label in classes:
        pmi = calculate_pmi(word, class_label, dataset)
        print(f"PMI for '{word}' and '{class_label}': {pmi}")


0.0 0.6 0.0
PMI for 'loved' and 'positive': 0
0.0 0.4 0.0
PMI for 'loved' and 'negative': 0
0.0 0.6 0.0
PMI for 'acting' and 'positive': 0
0.0 0.4 0.0
PMI for 'acting' and 'negative': 0


In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 57.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 90.3 MB/s eta 0:00:00


In [2]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.7/214.7 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.9 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=ed03477417a3fbf2034b850e7c26de3ab281395f7036183cf6e84b1757de638b
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [3]:
import pandas as pd
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification,Trainer, TrainingArguments
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from tqdm import tqdm
import wandb
import os

In [4]:
from google.colab import drive
drive.mount('/content/drive')

dataset = pd.read_csv("/content/drive/MyDrive/NLP RES DATASET/labeled_dataHSOLemotion.csv")

dataset[["text","label"]] = dataset[["tweet","emotion"]]

dataset = dataset[["text","label"]]

Mounted at /content/drive


In [18]:
from sklearn.model_selection import train_test_split


train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=25)

print(f"No. of training examples: {train_data.shape[0]}")
print(f"No. of testing examples: {test_data.shape[0]}")

No. of training examples: 19826
No. of testing examples: 4957


In [19]:
train_data

,text,label
20694,Sexy ass yellow bone,1.0
5697,@catleesh_ I have small pussy size plugs,1.0
19767,RT @nostalgic_mike: Aw y'all some hoes hating ...,3.0
22656,"When Chief Keef said ""I aint done turning up b...",1.0
9848,Hoes always tryna say they look like a celebri...,3.0
...,...,...
21245,"Sweet baby birds, you crack me up. ~~ Houston ...",0.0
23468,"You're irrelevant, bitch",1.0
1175,&#8220;@Cheddar85: Got the carburetor foyer my...,0.0
2934,@DS_Arrogance @HG_Shit I bet I can pull at lea...,1.0


In [20]:
# load model and tokenizer and define length of the text sequence
model = RobertaForSequenceClassification.from_pretrained('roberta-base')
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base', max_length = 512)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should pr

In [21]:
# define a function that will tokenize the model, and will return the relevant inputs for the model
def tokenization(batched_text):
    return tokenizer(batched_text['text'], padding = True, truncation=True)


train_data = train_data.map(tokenization)
test_data = test_data.map(tokenization)

AttributeError: ignored

In [ ]:
train_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

In [ ]:
# define accuracy metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
# define the training arguments
training_args = TrainingArguments(
    output_dir = '/media/data_files/github/website_tutorials/results',
    num_train_epochs=3,
    per_device_train_batch_size = 4,
    gradient_accumulation_steps = 16,
    per_device_eval_batch_size= 8,
    evaluation_strategy = "epoch",
    disable_tqdm = False,
    load_best_model_at_end=True,
    warmup_steps=500,
    weight_decay=0.01,
    logging_steps = 8,
    fp16 = True,
    logging_dir='/media/data_files/github/website_tutorials/logs',
    dataloader_num_workers = 0,
    run_name = 'roberta-classification_titan'
)

In [ ]:
# instantiate the trainer class and check for available devices
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=test_data
)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

In [ ]:
# train the model
trainer.train()

In [ ]:
trainer.evaluate()